In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2,3"

%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib as mpl

from argparse import Namespace

import torch
import requests
import numpy as np

import cv2
from PIL import Image

import utils_model
import utils_gradio
import utils_attn


# Update rcParams to disable axes
mpl.rcParams['axes.spines.left'] = False
mpl.rcParams['axes.spines.right'] = False
mpl.rcParams['axes.spines.top'] = False
mpl.rcParams['axes.spines.bottom'] = False
mpl.rcParams['xtick.bottom'] = False
mpl.rcParams['xtick.top'] = False
mpl.rcParams['xtick.labelbottom'] = False
mpl.rcParams['ytick.left'] = False
mpl.rcParams['ytick.right'] = False
mpl.rcParams['ytick.labelleft'] = False


In [2]:
model_name_or_path = "microsoft/Phi-3.5-vision-instruct" 
# model_name_or_path = "llava-hf/llava-1.5-7b-hf"

args = Namespace(
    model_name_or_path=model_name_or_path,
    load_4bit=False,
    load_8bit=False,
)
processor, model = utils_model.get_processor_model(args)

utils_gradio.processor = processor
utils_gradio.model = model

The repository for microsoft/Phi-3.5-vision-instruct contains custom code which must be executed to correctly load the model. You can inspect the repository content at https://hf.co/microsoft/Phi-3.5-vision-instruct.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N]  y


/home/shaoyent/mambaforge/envs/gradio_lvlm/lib/python3.10/site-packages/transformers/models/auto/image_processing_auto.py:590: FutureWarning: The image_processor_class argument is deprecated and will be removed in v4.42. Please use `slow_image_processor_class`, or `fast_image_processor_class` instead
  warnings.warn(


The repository for microsoft/Phi-3.5-vision-instruct contains custom code which must be executed to correctly load the model. You can inspect the repository content at https://hf.co/microsoft/Phi-3.5-vision-instruct.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N]  y
The repository for microsoft/Phi-3.5-vision-instruct contains custom code which must be executed to correctly load the model. You can inspect the repository content at https://hf.co/microsoft/Phi-3.5-vision-instruct.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N]  y


Loading checkpoint shards: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:02<00:00,  1.37s/steps]


In [19]:
prompt = "<|image_{i}|>\nWhat's unusual?"
# image_file = "http://images.cocodataset.org/val2017/000000039769.jpg"
# raw_image = Image.open(requests.get(image_file, stream=True).raw)
raw_image = Image.open('extreme_ironing.jpg')

image_process_mode = "Default"
state, _, _, _ = utils_gradio.add_text(None, prompt, raw_image, image_process_mode)

In [20]:
temperature = 0.2
top_p = 0.7
max_new_tokens = 16

state, _ = utils_gradio.lvlm_bot(state, temperature, top_p, max_new_tokens)

recovered_image = state.recovered_image
fig = plt.figure()
plt.imshow(recovered_image)
role, message = state.messages[-1]
print(f'{role}: {message}')

AssertionError: total images must be the same as the number of image tags, got 0 image tags and 1 images

In [ ]:

def show_cam_on_image(img: np.ndarray,
                      mask: np.ndarray,
                      colormap: int = cv2.COLORMAP_JET) -> np.ndarray:
    """ This function overlays the cam mask on the image as an heatmap.
    By default the heatmap is in BGR format.
    :param img: The base image in RGB or BGR format.
    :param mask: The cam mask.
    :param use_rgb: Whether to use an RGB or BGR heatmap, this should be set to True if 'img' is in RGB format.
    :param colormap: The OpenCV colormap to be used.
    :returns: The default image with the cam overlay.
    """
    heatmap = cv2.applyColorMap(np.uint8(255 * mask), colormap)
    heatmap = cv2.cvtColor(heatmap, cv2.COLOR_BGR2RGB)
    heatmap = np.float32(heatmap) / 255

    if np.max(img) > 1:
        raise Exception(
            "The input image should np.float32 in the range [0, 1]")

    cam = heatmap + img
    cam = cam / np.max(cam)
    return np.uint8(255 * cam)

In [ ]:
type_selector = 'all'
fn_attention = state.attention_key + '_relevancy.pt'
recovered_image = state.recovered_image
img_idx = state.image_idx
word_rel_maps = torch.load(fn_attention)

word_rel_map = word_rel_maps[type_selector]
image_list = []


fig, axes = plt.subplots(4, 4, figsize=(10,10))
# fig.tight_layout()

for ax, (rel_key, rel_map) in zip(axes.ravel(), word_rel_map.items()):

    if (rel_map.shape[-1] != 577):
        rel_map = rel_map[-1,:][img_idx:img_idx+576].reshape(1,1,24,24).float()
    else:
        rel_map = rel_map[0,1:].reshape(1,1,24,24).float()
        
    rel_map_upscaled = torch.nn.functional.interpolate(rel_map, scale_factor=14, mode='bilinear')
    rel_map_upscaled = (rel_map_upscaled - rel_map_upscaled.min()) / (rel_map_upscaled.max() - rel_map_upscaled.min())
    rel_map_upscaled = rel_map_upscaled.squeeze(0).squeeze(0)
    
    original_image = np.array(recovered_image)
    original_image = (original_image - original_image.min()) / (original_image.max() - original_image.min())
    vis = show_cam_on_image(original_image, rel_map_upscaled.cpu().numpy())
    # vis = Image.from_array(vis)
    
    ax.imshow(vis)
    ax.set_title(f'{rel_key}')

In [ ]:
state.__dict__.keys()

In [ ]:
word_rel_maps['llama_token'].keys()

In [ ]:
word_rel_maps.keys()

In [ ]:
from transformers import AutoModelForCausalLM, AutoProcessor
model_id = "microsoft/Phi-3.5-vision-instruct" 

# Note: set _attn_implementation='eager' if you don't have flash_attn installed
model = AutoModelForCausalLM.from_pretrained(
  model_id, 
  device_map="cuda", 
  trust_remote_code=True, 
  torch_dtype="auto", 
  _attn_implementation='eager'    
)

# for best performance, use num_crops=4 for multi-frame, num_crops=16 for single-frame.
processor = AutoProcessor.from_pretrained(model_id, 
  trust_remote_code=True, 
  num_crops=4
) 

In [ ]:
model.model.vision_embed_tokens.img_processor.config.output_attentions = True

In [ ]:
all([hasattr(getattr(model, i) for i in 'model.vision_embed_tokens'.split('.')])

In [15]:
processor.tokenizer.__dict__.keys()

dict_keys(['legacy', 'add_prefix_space', '_tokenizer', '_decode_use_source_tokenizer', 'init_inputs', 'init_kwargs', 'name_or_path', '_processor_class', 'model_max_length', 'padding_side', 'truncation_side', 'model_input_names', 'clean_up_tokenization_spaces', 'split_special_tokens', 'deprecation_warnings', '_in_target_context_manager', 'chat_template', '_pad_token_type_id', 'verbose', '_special_tokens_map', 'extra_special_tokens', 'SPECIAL_TOKENS_ATTRIBUTES', '_add_bos_token', '_add_eos_token', 'use_default_system_prompt', 'vocab_file'])